In [ ]:
%matplotlib widget

import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import matplotlib.pyplot as plt

from python.fluorophores import FlBleach
from python.simulators import SimSequencefile

In [ ]:
sim = SimSequencefile()
sim.posgalvo = [0, 0, 0]
sim.posEOD = [0, 0, 0]

laserpower = 1
fl = FlBleach()  # define a bleaching fluorophore
fl.photonbudget = 200000
fl.pos = [200, 50, 0]
fl.brightness = 300  # kHz 
sim.fluorophores = fl
sim.background = 0  # a background that is not matched with a proper estimate leads to 'tails'
sim.background_estimated = 0  # a similar result is obtained when no background is present but 
                              # the background is underestimatedd (negative background estimate). 
                              # This leads to a bias in the estimator and appearance of "tails"
                              # over-estimation of background leads to instabilities

In [ ]:
fname = os.path.join(SCRIPT_DIR, "examples", "Tracking_2D.json")
fname2 = os.path.join(SCRIPT_DIR, "settings", "PSFvectorial2D.json") # use a PSF that is defined via a json file
sim.loadsequence(fname, fname2)

In [ ]:
# test different confounding factors, otherwise comment out:
sim.psfvec.setpar(**{'beadradius': 0*50e-9})  # in m can also lead to tails. Set to zero if no bead used
sim.sequence['locLimit'] = 100  # only track for 1000 localizations
sim.makepatterns()
out = sim.runSequence(repetitions=1)

In [ ]:
plt.figure()
plt.plot(out.loc.loccounter, out.loc.xnm)
plt.plot(out.loc.loccounter, out.loc.xfl1)
plt.plot(out.loc.loccounter, out.loc.xgalvo)
plt.plot(out.loc.loccounter, out.loc.xeod)
plt.xlabel('time (itr)')
plt.ylabel('x position(nm)')
plt.legend(['estimated', 'fluorophore','xgalvo','EOD'])
indf = (out.loc.itr==max(out.loc.itr)) & (out.loc.vld==1)
sim.summarize_results(out, filter=indf.squeeze())  # display summary of simulation